# Decision Trees - Titanic Survival Classification

**Objective**: To classify if the passenger is dead (0) or survived (1).

**Outline**:

1. **Constructing the Model** 
    - 1.1 Decision Tree (DT) 
     
2. **Data Preprocessing**
3. **Model Evaluation**

Dataset from https://www.kaggle.com/datasets/yasserh/titanic-dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

SEED = 42 # set the random seed to 42 for reproducibility

## 1. Constructing the Model

**Model Outline**:
  1. **\_\_init\_\_** -> for initializing the hyperparameters
  2. **fit** -> to fit X_train and y_train into the model to train
  3. **predict** -> return the predicted class

In [2]:
class DecisionTree:
    # public:
    def __init__(self, max_depth):


        """
        One hyperparameter in THIS dt:
        1. max_depth -> the maximum depth of the tree

        # dt has a high variance, so it technically needs at least one regularization method to avoid bad performance
        # some other regularization methods/ hyperparameters can be -> minimum samples per leaf, etc.

        """


        self.max_depth = max_depth
        self.trained = False

    def fit(self, X, y):


        """
        fit/ train the model:
        1. initialize (get the shape of X)
        2. start splitting recursively on the root node and a dt is formed

        """


        self._initialize(X)
        self.root_node = self._split(X, y, depth=0)  # tree is built on the root node
        self.trained = True

    def predict(self, X):
        y_pred = []

        if self.trained:
            for sample in X:
                node = self.root_node  # reset the node to root
                while not node.is_leaf:
                    if sample[node.feature] < node.threshold:  # choose feature_j to meet the threshold
                        node = node.left_node
                    else:
                        node = node.right_node

                y_pred.append(node.predicted_class)

        return np.array(y_pred)

    # private:
    def _initialize(self, X):
        m_samples, n_features = X.shape

        self.m_samples = m_samples
        self.n_features = n_features

    def _split(self, X, y, depth):

        """
        dt is a recursive algorithm, so we need to set the terminal conditions for it to stop:
        1. max_depth is met 
        2. y is pure, i.e. no more misclassifications/ only one pred_class in each node



        edge case, which we also need to set a terminal condition for it to stop:
        1. 

        """

        
        if depth == self.max_depth or self._is_pure(y):  # terminal condition
            return Node(is_leaf=True, predicted_class=self._majority_class(y))
        
        elif len(y) == 0: # edge case
            return Node(is_leaf=True, predicted_class=None)
        
        """
        dt is also considered as a greedy and bruteforcing algorithm,
        it finds the local optimum at its current step by going through all the combinations of features being the thresholds
        1. 
        2.
        3.

        """

        best_threshold = None
        best_feature = None
        best_information_gain = -np.inf

        # bruteforce
        for jth_feature in range(self.n_features):
            values = np.unique(X[:, jth_feature])  # get the unique values of each feature

            for t_threshold in values:
                information_gain = self._information_gain(X, y, jth_feature, t_threshold)
                if information_gain > best_information_gain:
                    best_threshold = t_threshold
                    best_feature = jth_feature
                    best_information_gain = information_gain

        if best_information_gain == -np.inf:
            return Node(is_leaf=True, predicted_class=self._majority_class(y))

        left_index = X[:, best_feature] < best_threshold
        right_index = X[:, best_feature] >= best_threshold

        left_node = self._split(X[left_index, :], y[left_index], depth + 1)
        right_node = self._split(X[right_index, :], y[right_index], depth + 1)

        return Node(left_node, right_node, best_feature, best_threshold)

    def _information_gain(self, X_node, y_node, jth_feature, t_threshold):

        """
        
        """

        parent_node = y_node
        total_entropy = self._cross_entropy(parent_node)

        left_node = parent_node[X_node[:, jth_feature] < t_threshold]
        right_node = parent_node[X_node[:, jth_feature] >= t_threshold]

        left_proportion, left_entropy = len(left_node) / len(parent_node), self._cross_entropy(left_node)
        right_proportion, right_entropy = len(right_node) / len(parent_node), self._cross_entropy(right_node)

        information_gain = total_entropy - (left_proportion * left_entropy + right_proportion * right_entropy)

        return information_gain

    def _cross_entropy(self, y_node):
        if len(y_node) == 0:  # avoid zero division error
            return 0

        proportions = np.bincount(y_node) / len(y_node)
        proportions = proportions[proportions > 0]

        entropy = -np.sum(proportions * np.log(proportions))

        return entropy

    def _is_pure(self, y):
        return len(np.unique(y)) == 1

    def _majority_class(self, y):
        return np.argmax(np.bincount(y))


class Node:
    def __init__(self, left_node=None, right_node=None, feature=None, threshold=None, is_leaf=False, predicted_class=None):
        self.left_node = left_node
        self.right_node = right_node

        self.feature = feature
        self.threshold = threshold

        self.is_leaf = is_leaf
        self.predicted_class = predicted_class

## 2. Data Preprocessing

In [3]:
df = pd.read_csv("titanic_data.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Fare", "Cabin", "Age"], inplace=True)
df

,Survived,Pclass,Sex,SibSp,Parch,Embarked
0,0,3,male,1,0,S
1,1,1,female,1,0,C
2,1,3,female,0,0,S
3,1,1,female,1,0,S
4,0,3,male,0,0,S
...,...,...,...,...,...,...
886,0,2,male,0,0,S
887,1,1,female,0,0,S
888,0,3,female,1,2,S
889,1,1,male,0,0,C


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  891 non-null    int64 
 1   Pclass    891 non-null    int64 
 2   Sex       891 non-null    object
 3   SibSp     891 non-null    int64 
 4   Parch     891 non-null    int64 
 5   Embarked  889 non-null    object
dtypes: int64(4), object(2)
memory usage: 41.9+ KB


In [6]:
df.dropna(axis=0, inplace=True)

df[["Pclass", "Parch"]] = df[["Pclass", "Parch"]].astype(object)
df = pd.get_dummies(df)
df

C:\Users\user\AppData\Local\Temp\ipykernel_22128\3401344964.py:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df)
C:\Users\user\AppData\Local\Temp\ipykernel_22128\3401344964.py:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df)


,Survived,SibSp,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1
1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0
2,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1
887,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1
888,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1
889,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0


In [7]:
X = df.drop(columns="Survived").to_numpy()
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

## 3. Model Evaluation

In [8]:
clf = DecisionTree(4)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

my_model_score = accuracy_score(y_test, y_pred)
print(f"The accuracy score of my model is {my_model_score}")

The accuracy score of my model is 0.8014981273408239


In [9]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=SEED)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

sklearn_model_score = accuracy_score(y_test, y_pred)
print(f"The accuracy score of sklearn's model is {sklearn_model_score}")

The accuracy score of sklearn's model is 0.7865168539325843
